In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
pd.set_option('display.max_columns',40)
from google.colab import drive
import os
os.chdir('/content/drive/MyDrive/Real_Trader_EDA_R1/Indian_Stocks_5Nov24/Updated_data/Updated_ES')

In [3]:
# Path to your data folder in Google Drive
data_folder = '/content/drive/MyDrive/Real_Trader_EDA_R1/Indian_Stocks_5Nov24/Updated_data/Updated_ES'
# List files in the specified folder to confirm access
folder_path = os.path.join(data_folder)  # Update 'path_to_your_data_folder' with your actual folder path
file_list = os.listdir(folder_path)
file_list

['complete_data_2019_30sept2024(OHLC).csv',
 'Final_Updated_logic_Updated_ES_25NOV.csv',
 'Final_Updated_logic_Updated_ES_ESDT_25NOV24.csv']

In [4]:
s = pd.read_csv("Final_Updated_logic_Updated_ES_ESDT_25NOV24.csv")

In [5]:
s.columns

Index(['Time Stamp', 'Open', 'High', 'Low', 'Close', 'SellersLevel4',
       'SellersLevel3', 'SellersLevel2', 'SellersLevel1', 'SellersLevel0',
       'BuyersLevel0', 'BuyersLevel1', 'BuyersLevel2', 'BuyersLevel3',
       'BuyersLevel4', 'BuyersInControlPrice', 'SellersInControlPrice',
       'Entry_Type', 'Entry', 'Defence', 'Stop', 'Target'],
      dtype='object')

In [6]:
import pandas as pd
from tqdm import tqdm
import os

def apply_trading_logic(df):
    # Initialize the new columns with 0
    df['Actual_Profit'] = 0
    df['D_Profit'] = 0
    df['Actual_Loss'] = 0
    df['Ideal_Target_Buyer'] = 0
    df['Defence_Target_Buyer'] = 0
    df['Ideal_Target_Seller'] = 0
    df['Defence_Target_Seller'] = 0
    df['Defence_Target'] = 0
    df['Ideal_Target'] = 0
    df['Ideal_Target_Agressive'] = 0

    # Iterate over each row with tqdm for progress tracking
    for i in tqdm(range(len(df))):
        row = df.iloc[i]

        # Entry Value Check
        if row['Entry'] != 0:
            entry = row['Entry']
            defence = row['Defence']

            # Buyers Logic
            if row['BuyersInControlPrice'] != 0:
                # Defence Check
                low_future_rows = df.iloc[i+1:]['Low']
                matched_low_index = low_future_rows[low_future_rows == defence].index
                if not matched_low_index.empty:
                    matched_low_index = matched_low_index[0]
                    max_high = df.loc[i:matched_low_index, 'High'].max()
                    max_close = df.loc[i:matched_low_index, 'Close'].max()

                    df.loc[i, 'Ideal_Target_Buyer'] = max_close
                    df.loc[i, 'Ideal_Target_Agressive'] = max_high

                    # Target vs. Max High Value Comparison
                    if row['Target'] <= max_high:
                        df.loc[i, 'Actual_Profit'] = row['Target'] - entry
                    else:
                        stop = row['Stop']
                        if stop != 0:
                            df.loc[i, 'Defence_Target_Buyer'] = ((entry + defence) / 2) + 0.25
                            matched_stop_index = low_future_rows[low_future_rows == stop].index
                            if not matched_stop_index.empty:
                                matched_stop_index = matched_stop_index[0]
                                max_high_stop = df.loc[i:matched_stop_index, 'High'].max()
                                max_close_stop = df.loc[i:matched_stop_index, 'Close'].max()

                                # Defence Target vs. Max High Value Comparison
                                if df.loc[i, 'Defence_Target_Buyer'] <= max_high_stop:
                                    df.loc[i, 'D_Profit'] = df.loc[i, 'Defence_Target_Buyer'] - entry
                                else:
                                    df.loc[i, 'Actual_Loss'] = df.loc[i, 'Defence_Target_Buyer'] - entry

            # Sellers Logic
            if row['SellersInControlPrice'] != 0:
                # Defence Check
                high_future_rows = df.iloc[i+1:]['High']
                matched_high_index = high_future_rows[high_future_rows == defence].index
                if not matched_high_index.empty:
                    matched_high_index = matched_high_index[0]
                    min_low = df.loc[i:matched_high_index, 'Low'].min()
                    min_close = df.loc[i:matched_high_index, 'Close'].min()

                    df.loc[i, 'Ideal_Target_Seller'] = min_close
                    df.loc[i, 'Ideal_Target_Agressive'] = min_low

                    # Target vs. Min Low Value Comparison
                    if row['Target'] >= min_low:
                        df.loc[i, 'Actual_Profit'] = entry - row['Target']
                    else:
                        stop = row['Stop']
                        if stop != 0:
                            df.loc[i, 'Defence_Target_Seller'] = ((entry + defence) / 2) - 0.25
                            matched_stop_index = high_future_rows[high_future_rows == stop].index
                            if not matched_stop_index.empty:
                                matched_stop_index = matched_stop_index[0]
                                min_low_stop = df.loc[i:matched_stop_index, 'Low'].min()
                                min_close_stop = df.loc[i:matched_stop_index, 'Close'].min()

                                # Defence Target vs. Min Low Value Comparison
                                if df.loc[i, 'Defence_Target_Seller'] >= min_low_stop:
                                    df.loc[i, 'D_Profit'] = entry - df.loc[i, 'Defence_Target_Seller']
                                else:
                                    df.loc[i, 'Actual_Loss'] = df.loc[i, 'Defence_Target_Seller'] - entry

        # Additional Calculations
        df.loc[i, 'Ideal_Target'] = df.loc[i, 'Ideal_Target_Buyer'] + df.loc[i, 'Ideal_Target_Seller']
        df.loc[i, 'Defence_Target'] = df.loc[i, 'Defence_Target_Buyer'] + df.loc[i, 'Defence_Target_Seller']

    # Drop unnecessary columns if they exist
    columns_to_drop = ["Ideal_Target_Buyer", "Ideal_Target_Seller", "Defence_Target_Buyer", "Defence_Target_Seller"]
    df.drop(columns=[col for col in columns_to_drop if col in df.columns], inplace=True)

    return df

# def process_files(input_directory, output_directory):
#     # Ensure the output directory exists
#     os.makedirs(output_directory, exist_ok=True)

#     # Process each CSV file in the input directory
#     for filename in os.listdir(input_directory):
#         if filename.endswith('.csv'):
#             input_path = os.path.join(input_directory, filename)
#             # Add prefix "Complete_" to the output file name
#             output_path = os.path.join(output_directory, f"Complete_{filename}")

#             # Read the input file
#             df = pd.read_csv(input_path)

#             # Convert 'Time Stamp' to datetime
#             if 'Time Stamp' in df.columns:
#                 df['Time Stamp'] = pd.to_datetime(df['Time Stamp'], errors='coerce')

#             # Drop the "Unnamed: 0" column if it exists
#             if 'Unnamed: 0' in df.columns:
#                 df.drop(columns=['Unnamed: 0'], inplace=True)

#             # Apply the trading logic
#             df = apply_trading_logic(df)

#             # Save the processed file to the output directory
#             df.to_csv(output_path, index=False)
#             print(f"Processed {filename} and saved to {output_path}")


# # Example usage
# output_directory = '/content/drive/MyDrive/Real_Trader_EDA_R1/ES_5min_data/Calculated_Data_Folders/Complete_Data_Files_ideal_Aggressive_with_duplicates'
# process_files(data_folder, output_directory)

In [7]:
final_T_AT = apply_trading_logic(s)

  0%|          | 0/367484 [00:00<?, ?it/s]<ipython-input-6-de606f4fb500>:70: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '3237.25' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[i, 'Ideal_Target_Agressive'] = min_low
<ipython-input-6-de606f4fb500>:74: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '3.25' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[i, 'Actual_Profit'] = entry - row['Target']
<ipython-input-6-de606f4fb500>:69: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '3240.75' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[i, 'Ideal_Target_Seller'] = min_close
<ipython-input-6-de606f4fb500>:78

In [8]:
final_T_AT.columns

Index(['Time Stamp', 'Open', 'High', 'Low', 'Close', 'SellersLevel4',
       'SellersLevel3', 'SellersLevel2', 'SellersLevel1', 'SellersLevel0',
       'BuyersLevel0', 'BuyersLevel1', 'BuyersLevel2', 'BuyersLevel3',
       'BuyersLevel4', 'BuyersInControlPrice', 'SellersInControlPrice',
       'Entry_Type', 'Entry', 'Defence', 'Stop', 'Target', 'Actual_Profit',
       'D_Profit', 'Actual_Loss', 'Defence_Target', 'Ideal_Target',
       'Ideal_Target_Agressive'],
      dtype='object')

In [ ]:
# final_T_AT[['Dist_Bt_T_IT', 'Dist_Bt_T_IAT']].sample(50)

In [9]:
final_T_AT.drop(columns=['Actual_Profit','D_Profit', 'Actual_Loss', 'Defence_Target'],axis=1,inplace=True)

In [ ]:
import numpy as np

# Given values
values = [4627.75, 4632.75, 4627.25, 4632]

# Calculate the median
median_value = np.median(values)
median_value

4629.875

In [10]:
final_T_AT['Dist_Bt_T_IT'] = (final_T_AT['Ideal_Target'] - final_T_AT['Target'])
final_T_AT['Dist_Bt_T_IAT'] = (final_T_AT['Ideal_Target_Agressive'] - final_T_AT['Target'])


In [11]:
final_T_AT.columns

Index(['Time Stamp', 'Open', 'High', 'Low', 'Close', 'SellersLevel4',
       'SellersLevel3', 'SellersLevel2', 'SellersLevel1', 'SellersLevel0',
       'BuyersLevel0', 'BuyersLevel1', 'BuyersLevel2', 'BuyersLevel3',
       'BuyersLevel4', 'BuyersInControlPrice', 'SellersInControlPrice',
       'Entry_Type', 'Entry', 'Defence', 'Stop', 'Target', 'Ideal_Target',
       'Ideal_Target_Agressive', 'Dist_Bt_T_IT', 'Dist_Bt_T_IAT'],
      dtype='object')

In [12]:
final_T_AT[['Dist_Bt_T_IT', 'Dist_Bt_T_IAT']].sample(50)

,Dist_Bt_T_IT,Dist_Bt_T_IAT
280328,0.00,0.00
19671,0.00,0.00
328824,0.00,0.00
137593,-1.00,-0.25
160685,0.00,0.00
227139,0.00,0.00
259252,0.00,0.00
337557,0.00,0.00
313800,0.00,0.00
18598,0.00,0.00


In [13]:
final_T_AT.columns

Index(['Time Stamp', 'Open', 'High', 'Low', 'Close', 'SellersLevel4',
       'SellersLevel3', 'SellersLevel2', 'SellersLevel1', 'SellersLevel0',
       'BuyersLevel0', 'BuyersLevel1', 'BuyersLevel2', 'BuyersLevel3',
       'BuyersLevel4', 'BuyersInControlPrice', 'SellersInControlPrice',
       'Entry_Type', 'Entry', 'Defence', 'Stop', 'Target', 'Ideal_Target',
       'Ideal_Target_Agressive', 'Dist_Bt_T_IT', 'Dist_Bt_T_IAT'],
      dtype='object')

In [ ]:
final_T_AT.to_csv("Final_T_IAT_Finnhub_23oct24.csv",index=False)

In [ ]:
final_T_AT.loc[final_T_AT['Ideal_Target'] == 0, 'Ideal_Target'] = final_T_AT['Target']
final_T_AT.loc[final_T_AT['Ideal_Target_Agressive'] == 0, 'Ideal_Target_Agressive'] = final_T_AT['Ideal_Target']


In [ ]:
# Filter the DataFrame to show only rows where 'Target' != 0
final_T_AT[(final_T_AT['Target'] != 0) & (final_T_AT['Ideal_Target'] == 0)]

,Time Stamp,Open,High,Low,Close,SellersLevel4,SellersLevel3,SellersLevel2,SellersLevel1,SellersLevel0,BuyersLevel0,BuyersLevel1,BuyersLevel2,BuyersLevel3,BuyersLevel4,BuyersInControlPrice,SellersInControlPrice,Entry_Type,Entry,Defence,Stop,Target,Ideal_Target,Ideal_Target_Agressive,Dist_Bt_T_IT,Dist_Bt_T_IAT


In [14]:
final_T_AT.drop(["Entry_Type"],axis=1,inplace=True)

In [15]:
final_T_AT.to_csv("/content/drive/MyDrive/Real_Trader_EDA_R1/Indian_Stocks_5Nov24/Updated_data/Updated_ES/Processed_Complet_IAT_Updated_Data_Updated_ES_25NOV24.csv",index=False)

In [ ]:
_it[(s_it['Target'] != 0) & (s_it['Ideal_Target'] == 0)]

In [ ]:
s_it.loc[s_it['Ideal_Target'] == 0, 'Ideal_Target'] = s_it['Target']

In [ ]:
s_it.loc[s_it['Ideal_Target_Agressive'] == 0, 'Ideal_Target_Agressive'] = s_it['Ideal_Target']

In [ ]:
final_T_AT.to_csv("/content/drive/MyDrive/Real_Trader_EDA_R1/ES_5min_data/Calculated_Data_Folders/Complete_Data_Files_ideal_Aggressive_without_duplicates/Processed_ESDT_ZB 09-24_Minute_5.csv",index=False)

In [ ]:
complete_T_IT = apply_trading_logic(s)

  0%|          | 0/70558 [00:00<?, ?it/s]<ipython-input-14-ae8a1401be9a>:72: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '4580.75' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[i, 'Ideal_Target_Agressive'] = min_low
<ipython-input-14-ae8a1401be9a>:76: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '3.5' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[i, 'T_Profit'] = entry - row['Target']
<ipython-input-14-ae8a1401be9a>:96: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '35.25' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[i, 'Dist_Bt_T_It'] = abs(df.loc[i, 'Ideal_Target'] - row['Target'])
<ipython-input-14-ae8a1401be9a>:97: Future

In [ ]:
complete_T_IT.columns

Index(['Time Stamp', 'Open', 'High', 'Low', 'Close', 'SellersLevel4',
       'SellersLevel3', 'SellersLevel2', 'SellersLevel1', 'SellersLevel0',
       'BuyersLevel0', 'BuyersLevel1', 'BuyersLevel2', 'BuyersLevel3',
       'BuyersLevel4', 'BuyersInControlPrice', 'SellersInControlPrice',
       'Entry', 'Defence', 'Stop', 'Target', 'T_Profit', 'D_Profit', 'Loss',
       'Defence_Target', 'Ideal_Target', 'Ideal_Target_Agressive',
       'Dist_Bt_T_It', 'Dist_Bt_T_IAT'],
      dtype='object')

In [ ]:
complete_T_IT.to_csv("/content/drive/MyDrive/Real_Trader_EDA_R1/ES_5min_data/Calculated_Data_Folders/Complete_Data_T_IT_P_L/Actual_target_loss.csv",index=False)

In [ ]:
import pandas as pd
from tqdm import tqdm
import os

def apply_trading_logic(df):
    # Initialize the new columns with 0
    df['T_Profit'] = 0
    df['D_Profit'] = 0
    df['Loss'] = 0
    df['Ideal_Target_Buyer'] = 0
    df['Defence_Target_Buyer'] = 0
    df['Ideal_Target_Seller'] = 0
    df['Defence_Target_Seller'] = 0
    df['Defence_Target'] = 0
    df['Ideal_Target'] = 0
    df['Ideal_Target_Agressive'] = 0
    df['Dist_Bt_T_It'] = 0
    df['Dist_Bt_T_IAT'] = 0

    # Iterate over each row with tqdm for progress tracking
    for i in tqdm(range(len(df))):
        row = df.iloc[i]

        # Entry Value Check
        if row['Entry'] != 0:
            entry = row['Entry']
            defence = row['Defence']

            # Buyers Logic
            if row['BuyersInControlPrice'] != 0:
                # Defence Check
                low_future_rows = df.iloc[i+1:]['Low']
                matched_low_index = low_future_rows[low_future_rows == defence].index
                if not matched_low_index.empty:
                    matched_low_index = matched_low_index[0]
                    max_high = df.loc[i:matched_low_index, 'High'].max()
                    max_close = df.loc[i:matched_low_index, 'Close'].max()

                    df.loc[i, 'Ideal_Target_Buyer'] = max_close
                    df.loc[i, 'Ideal_Target_Agressive'] = max_high

                    # Target vs. Max High Value Comparison
                    if row['Target'] <= max_high:
                        df.loc[i, 'T_Profit'] = row['Target'] - entry
                    else:
                        stop = row['Stop']
                        if stop != 0:
                            df.loc[i, 'Defence_Target_Buyer'] = ((entry + defence) / 2) + 0.25
                            matched_stop_index = low_future_rows[low_future_rows == stop].index
                            if not matched_stop_index.empty:
                                matched_stop_index = matched_stop_index[0]
                                max_high_stop = df.loc[i:matched_stop_index, 'High'].max()
                                max_close_stop = df.loc[i:matched_stop_index, 'Close'].max()

                                # Defence Target vs. Max High Value Comparison
                                if df.loc[i, 'Defence_Target_Buyer'] <= max_high_stop:
                                    df.loc[i, 'D_Profit'] = df.loc[i, 'Defence_Target_Buyer'] - entry
                                else:
                                    df.loc[i, 'Loss'] = df.loc[i, 'Defence_Target_Buyer'] - entry

            # Sellers Logic
            if row['SellersInControlPrice'] != 0:
                # Defence Check
                high_future_rows = df.iloc[i+1:]['High']
                matched_high_index = high_future_rows[high_future_rows == defence].index
                if not matched_high_index.empty:
                    matched_high_index = matched_high_index[0]
                    min_low = df.loc[i:matched_high_index, 'Low'].min()
                    min_close = df.loc[i:matched_high_index, 'Close'].min()

                    df.loc[i, 'Ideal_Target_Seller'] = min_close
                    df.loc[i, 'Ideal_Target_Agressive'] = min_low

                    # Target vs. Min Low Value Comparison
                    if row['Target'] >= min_low:
                        df.loc[i, 'T_Profit'] = entry - row['Target']
                    else:
                        stop = row['Stop']
                        if stop != 0:
                            df.loc[i, 'Defence_Target_Seller'] = ((entry + defence) / 2) - 0.25
                            matched_stop_index = high_future_rows[high_future_rows == stop].index
                            if not matched_stop_index.empty:
                                matched_stop_index = matched_stop_index[0]
                                min_low_stop = df.loc[i:matched_stop_index, 'Low'].min()
                                min_close_stop = df.loc[i:matched_stop_index, 'Close'].min()

                                # Defence Target vs. Min Low Value Comparison
                                if df.loc[i, 'Defence_Target_Seller'] >= min_low_stop:
                                    df.loc[i, 'D_Profit'] = entry - df.loc[i, 'Defence_Target_Seller']
                                else:
                                    df.loc[i, 'Loss'] = df.loc[i, 'Defence_Target_Seller'] - entry

        # Additional Calculations
        df.loc[i, 'Ideal_Target'] = df.loc[i, 'Ideal_Target_Buyer'] + df.loc[i, 'Ideal_Target_Seller']
        df.loc[i, 'Defence_Target'] = df.loc[i, 'Defence_Target_Buyer'] + df.loc[i, 'Defence_Target_Seller']
        df.loc[i, 'Dist_Bt_T_It'] = abs(df.loc[i, 'Ideal_Target'] - row['Target'])
        df.loc[i, 'Dist_Bt_T_IAT'] = abs(df.loc[i, 'Ideal_Target_Agressive'] - row['Target'])

    # Drop unnecessary columns if they exist
    columns_to_drop = ["Ideal_Target_Buyer", "Ideal_Target_Seller", "Defence_Target_Buyer", "Defence_Target_Seller"]
    df.drop(columns=[col for col in columns_to_drop if col in df.columns], inplace=True)

    return df

def process_files(input_directory, output_directory):
    # Ensure the output directory exists
    os.makedirs(output_directory, exist_ok=True)

    # Process each CSV file in the input directory
    for filename in os.listdir(input_directory):
        if filename.endswith('.csv'):
            input_path = os.path.join(input_directory, filename)
            # Add prefix "Complete_" to the output file name
            output_path = os.path.join(output_directory, f"Complete_{filename}")

            # Read the input file
            df = pd.read_csv(input_path)

            # Convert 'Time Stamp' to datetime
            if 'Time Stamp' in df.columns:
                df['Time Stamp'] = pd.to_datetime(df['Time Stamp'], errors='coerce')

            # Drop the "Unnamed: 0" column if it exists
            if 'Unnamed: 0' in df.columns:
                df.drop(columns=['Unnamed: 0'], inplace=True)

            # Apply the trading logic
            df = apply_trading_logic(df)

            # Save the processed file to the output directory
            df.to_csv(output_path, index=False)
            print(f"Processed {filename} and saved to {output_path}")


# Example usage
output_directory = '/content/drive/MyDrive/Real_Trader_EDA_R1/ES_5min_data/Calculated_Data_Folders/Final_Data_T_IT_Pro_Loss'
process_files(data_folder, output_directory)
